<a href="https://colab.research.google.com/github/sandeshchand87/NGram_LanguageModel/blob/main/QA_misteral.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://medium.com/emburse/question-answering-over-documents-e92658e7a405

In [ ]:
!pip install -U pypdf torch transformers langchain ipywidgets accelerate \
  sentence_transformers pyarrow pandas bitsandbytes einops xformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.1/286.1 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 96.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 812.8/812.8 kB 40.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.4/139.4 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 33.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.3/163.3 kB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 38.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 71.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.5/222.5 MB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━

In [ ]:
import os
os.environ['LC_ALL']='C.UTF-8'
os.environ['LANG']='C.UTF-8'

#drive.mount('/content/sample_data')
folder_path = '/content/sample_data/data'
print(folder_path)

/content/sample_data/data


In [ ]:
from langchain.document_loaders import PyPDFLoader
file = folder_path + "/Surround View Camera system.pdf"

loader = PyPDFLoader(folder_path + "/Surround View Camera system.pdf")
pages= loader.load_and_split()

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 512,
    chunk_overlap=50,
    separators=['\n\n', '\n', '(?=>\. )', ' ', '']
)
texts = text_splitter.split_documents(pages)

In [ ]:
print(len(pages), len(texts))

22 70


In [ ]:
print(pages[1].page_content)

Infront of the lens.  Generally, for a given sensor size, the shorter the focal length, th e wider the 
angle of view of the lens.  
 
 
Figure 2 Distorted Image produced by fish eye camera  
Camera calibration is the process of computing the extrinsic and intrinsic parameters of a 
camera. The extrinsic parameter such as  3*3 r otation matrix ,3*1 translation matrix and intrinsic 
parameters  are the camera parameter which is 3*3 matrix. The main goal is to find these 
parameters  and their corresponding image coordinates  (u, v). When we get the values of 
intrinsic and extrinsic parameters the camera is said to be calibrated.   
Once a camera is calibrated, we can use the image information to recover 3 -D information 
from 2 -D images. we can also undistort images taken with a fisheye camera . 
 
Figure 3 Distorted Fisheye image and its undistorted image after correct3 
The image coordinate system is the 2 -dimensional coordinate systems in pixel. So, the object 
has real world poin

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings

embedding= HuggingFaceEmbeddings(model_name = "sentence-transformers/all-mpnet-base-v2")

In [ ]:
from langchain.vectorstores import SKLearnVectorStore

vector_db_path =  "./document_vector_db.parquet"
vector_db = SKLearnVectorStore.from_documents(texts, embedding=embedding,persist_path= vector_db_path,
                                              serializer="parquet")


In [ ]:
print(vector_db.similarity_search("what is surroundview?"))

[Document(page_content='Surround View Camera -system  \nIntroduction:  \nThe Surround view system provides a complete 360-degree  view around the entire vehicles  \n(car, ship etc) with the help of number of cameras  (fisheye cameras) mounted on it.  The number \nof cameras is depending on the size and type of vehicles. Usually there are two types of \ncameras - Pinhole cameras and fish eye cameras. In this system, we are going to discuss on fish', metadata={'id': '20eab42a-ef72-478f-91b3-0222c9f410a1', 'source': '/content/sample_data/data/Surround View Camera system.pdf', 'page': 0}), Document(page_content='projection matrix. This matrix will convert the undistorted or rectified camera picture into bird \neye views of a rectangular area on the ground.  The projection matrices of these four cameras \nare not independent, and they must ensure that the projected areas can be assembled exactly.  \n \nFigure 14 Parameter settings for bird eye view9 \n \n9 https://github.com/neozhaoliang/su

In [ ]:
import pandas as pd
vector_db.persist()

df = pd.read_parquet(vector_db_path)

df

,ids,texts,metadatas,embeddings
0,20eab42a-ef72-478f-91b3-0222c9f410a1,Surround View Camera -system \nIntroduction: ...,"{'page': 0, 'source': '/content/sample_data/da...","[-0.02578999288380146, -0.08342663943767548, 0..."
1,564aeab1-0752-4c57-9b1d-57fd1efa495b,"eye cameras . As compare to Pinhole cameras, t...","{'page': 0, 'source': '/content/sample_data/da...","[-0.0085429847240448, -0.027984950691461563, 0..."
2,35fa209d-dc97-467f-8a90-2552febe0705,alignment corrects the fish eye distortion in ...,"{'page': 0, 'source': '/content/sample_data/da...","[-0.04039984196424484, -0.07470962405204773, 0..."
3,f1f7ecc2-80a1-4846-ae07-eb6dfd71628a,and color mismatch between adjacent views t...,"{'page': 0, 'source': '/content/sample_data/da...","[-0.012754177674651146, -0.08919082581996918, ..."
4,8156368d-7306-4a46-b03d-c52f0e97d29a,"but uses a special mapping (equisolid angle),...","{'page': 0, 'source': '/content/sample_data/da...","[-0.006576374638825655, -0.10711628943681717, ..."
...,...,...,...,...
65,f90ef2fc-67cc-4faa-a2a0-26f045aef904,pixel can easily detected. If the pixel falls ...,"{'page': 19, 'source': '/content/sample_data/d...","[-0.0328475758433342, -0.09125891327857971, -0..."
66,ec4169f7-be3d-4d56-a97d-7daa9bc56fd0,from 0 to 1. The fused (merge )image can be o...,"{'page': 20, 'source': '/content/sample_data/d...","[-0.02859026938676834, -0.10990551114082336, -..."
67,74689c10-cc83-4b20-91ac-adf752f84c41,"of different channel s (B, G, R) of the camera...","{'page': 20, 'source': '/content/sample_data/d...","[-0.00036798143992200494, -0.11691871285438538..."
68,92c5d61b-3603-4ff1-a7cf-909ff767fffd,Figure 29 Final image after bright ness+ colou...,"{'page': 21, 'source': '/content/sample_data/d...","[0.019129816442728043, -0.0633750930428505, -0..."


In [ ]:
import torch
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
from langchain import HuggingFacePipeline
from transformers import BitsAndBytesConfig


model = "tiiuae/falcon-7b-instruct"

device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

tokenizer= AutoTokenizer.from_pretrained(model)
'''
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16
)
'''
model = AutoModelForCausalLM.from_pretrained(
    model,
    trust_remote_code=True,
    load_in_4bit=True,
    device_map='auto'
)

model.eval()
generate_text = pipeline(task="text-generation", model=model, tokenizer=tokenizer,
                         trust_remote_code=True, max_new_tokens=100,
                         repetition_penalty=1.1, model_kwargs={"device_map": "auto",
                          "max_length": 1200, "temperature": 0.01, "torch_dtype":torch.bfloat16}
)
'''
generate_text = pipeline(
    model=model, tokenizer=tokenizer,
    return_full_text=True,  # langchain expects the full text
    task='text-generation',
    # we pass model parameters here too
    temperature=0.7,  # 'randomness' of outputs, 0.0 is the min and 1.0 the max
    max_new_tokens=512,  # mex number of tokens to generate in the output
    repetition_penalty=1.1  # without this output begins repeating
)
'''
hf_pipeline = HuggingFacePipeline(pipeline=generate_text)
print("model loaded")

cuda


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

model loaded


In [ ]:
from langchain.chains import RetrievalQA

qa = RetrievalQA.from_chain_type(llm=hf_pipeline, chain_type="stuff",
                                 retriever=vector_db.as_retriever(search_kwargs={"k": 3}),
                                 return_source_documents=True,
                                 verbose=False,
)

In [ ]:
qa

RetrievalQA(combine_documents_chain=StuffDocumentsChain(llm_chain=LLMChain(prompt=PromptTemplate(input_variables=['context', 'question'], template="Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\n{context}\n\nQuestion: {question}\nHelpful Answer:"), llm=HuggingFacePipeline(pipeline=<transformers.pipelines.text_generation.TextGenerationPipeline object at 0x7d829a6bab90>)), document_variable_name='context'), return_source_documents=True, retriever=VectorStoreRetriever(tags=['SKLearnVectorStore', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.sklearn.SKLearnVectorStore object at 0x7d830dfc3ac0>, search_kwargs={'k': 3}))

In [ ]:
query =  "tell me the hardware used in Surroundview project "
result = qa({"query":query})
print("Response :", result["result"]  )
print("source:",result['source_documents'][0].metadata['source'])
print("page:",int(result['source_documents'][0].metadata['page']+1))

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:391: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn('Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.')


Response : Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

Surround View Camera -system  
Introduction:  
The Surround view system provides a complete 360-degree  view around the entire vehicles  
(car, ship etc) with the help of number of cameras  (fisheye cameras) mounted on it.  The number 
of cameras is depending on the size and type of vehicles. Usually there are two types of 
cameras - Pinhole cameras and fish eye cameras. In this system, we are going to discuss on fish

time . 
We have two parts in this documentation 1. Camera calibration (whi ch we already explained) 
2. Perspective transformation.  
To implement this project on production environment, the proper hardware is used. We can use 
hardware like AGX Xavier  etc. As hardware selection process is in research phase so, we have 
not included i n this document.

Following steps can be used to obtain the fina

In [ ]:
#To implement this project on production environment, the proper hardware is used

query =  "which hardware is  used  in Surroundview project on production environment "
result = qa({"query":query})
print("Response :", result["result"]  )
print("source:",result['source_documents'][0].metadata['source'])
print("page:",int(result['source_documents'][0].metadata['page']+1))

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Response : Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

time . 
We have two parts in this documentation 1. Camera calibration (whi ch we already explained) 
2. Perspective transformation.  
To implement this project on production environment, the proper hardware is used. We can use 
hardware like AGX Xavier  etc. As hardware selection process is in research phase so, we have 
not included i n this document.

Surround View Camera -system  
Introduction:  
The Surround view system provides a complete 360-degree  view around the entire vehicles  
(car, ship etc) with the help of number of cameras  (fisheye cameras) mounted on it.  The number 
of cameras is depending on the size and type of vehicles. Usually there are two types of 
cameras - Pinhole cameras and fish eye cameras. In this system, we are going to discuss on fish

Following steps can be used to obtain the fina

In [ ]:
query =  "what are the parameters require for camera calibration process?"

result = qa({"query": query})

print("Response :", result["result"]  )
print("source:",result['source_documents'][0].metadata['source'])
print("page:",int(result['source_documents'][0].metadata['page']+1))

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Response : Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

During the calibration process, some parameters are discovered to fix most of the distortions 
caused by the camera . 
• Extrinsic parameters:  rotation and transl ation vectors that translate a 3 D points to 2 
D coordinates  
• Intrinsic parameters:  focal length, optical centre  and skew coefficients which is 
specific for each camera.  
• Radial distortion coefficients : causes straight line to appear curved. It occurs when

For camera  calibration , we can use calibration board (chessboard ) by holding it in our hand and 
take many images Infront of the camera. Same process will apply for each of the camera. The 
camera internal parameters and distortion coefficient can easily be obtained  by camer a 
calibration method which is discussed in camera calibration section above  (Steps for camera 
calibration and

In [ ]:
query =  "Define real world coordinate system with chess board patterns."
result = qa({"query":query})
print("Response :", result["result"]  )
print("source:",result['source_documents'][0].metadata['source'])
print("page:",int(result['source_documents'][0].metadata['page']+1))

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Response : Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

world coordinate is attached to the che ssboard and since all the corner points lie on a plane, we 
can arbitrarily choose Zw for every point to be 0. Since points are equally spaced in the 
chessboard, the (Xw, Yw) coordinates of each 3D point are easily de fined by taking one point 
as reference (0, 0) and defining remaining with respect to that reference point.  For X w, Yw 
values, we can simply pass the points as (0,0), (1,0), (2,0), ... which denotes the location of

we get the results in mm. (In this case, we don't know square size since we didn't take those 
images, so we pass in terms of square size)6. 
3D points are called  object points  and 2D image points are called  image points.  
 
Figure 8 real posit ion of chessboard  
Why is the chess board pattern so widely used in calibration?  
• Patterns are

In [ ]:
query =  "Why is the chessboard pattern so widely used in calibration?"
result = qa({"query":query})
print("Response :", result["result"]  )
print("source:",result['source_documents'][0].metadata['source'])
print("page:",int(result['source_documents'][0].metadata['page']+1))

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Response : Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

we get the results in mm. (In this case, we don't know square size since we didn't take those 
images, so we pass in terms of square size)6. 
3D points are called  object points  and 2D image points are called  image points.  
 
Figure 8 real posit ion of chessboard  
Why is the chess board pattern so widely used in calibration?  
• Patterns are distinct and easy to detect an image.  
• Corners of the square have sharp gradients in two direction and are ideal for 
localizing them.  
• Corners of the squares are at the intersection of chessboard lines7. 
 
Figure 9 Ches sboard and its internal squares  
Step2.  Capture multiple images of the checkerboard from different viewpoints  
 
6 https://docs.opencv.org/4.x/dc/dbb/tutorial_py_calibration.html  
 
7 https://learnopencv.com/camera -calibra tion-using -opencv/



In [ ]:
result

{'query': 'Why is the chessboard pattern so widely used in calibration?',
 'result': "Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\nwe get the results in mm. (In this case, we don't know square size since we didn't take those \nimages, so we pass in terms of square size)6. \n3D points are called  object points  and 2D image points are called  image points.  \n \nFigure 8 real posit ion of chessboard  \nWhy is the chess board pattern so widely used in calibration?  \n• Patterns are distinct and easy to detect an image.  \n• Corners of the square have sharp gradients in two direction and are ideal for \nlocalizing them.  \n• Corners of the squares are at the intersection of chessboard lines7. \n \nFigure 9 Ches sboard and its internal squares  \nStep2.  Capture multiple images of the checkerboard from different viewpoints  \n \n6 https://docs.opencv.org/4.x/dc/dbb/tutor

In [ ]:
query =  "how to calcualte each pixel of images in overlapping area."
result = qa({"query":query})
print("Response :", result["result"]  )
print("source:",result['source_documents'][0].metadata['source'])
print("page:",int(result['source_documents'][0].metadata['page']+1))

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Response : Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

Figure 25 Overlapping image after morphol ogical operation  
4. Find the  outer boundar ies of left  and front images lies outside the overlapping area projection 
image using  cv2.findcontour()  and cv.appro xPolyD P (for appro ximate polygon  outline ) 
 
Figure 26 Images o f non-overlapping  area (Front , left Image ) 
polyA  (the blue border in the upper left image)  is obtained by the contour after subtracting the 
overlapping area from the front camera  while polyB  (the green border in the upper right ima ge) 
is obtained after subtracting the overlapping area from the left camera . 
 
4.  Calculate the w eight of each pi xel in the overlapping area . First calculate  the distance  (dA, 
dB) of each pixel  to polygon  polyA and  polyB using cv2.pointPolygonTest . the corresponding 
weight of each pi xel can

In [ ]:
query =  "how to obtain the merge image of two  front and left image."
result = qa({"query":query})
print("Response :", result["result"]  )
print("source:",result['source_documents'][0].metadata['source'])
print("page:",int(result['source_documents'][0].metadata['page']+1))

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Response : Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

from 0 to 1.  The fused (merge )image can be obt ained as front image * G + ( 1-G) *left image 
where G is the weight matrix . 
6. Adjust the light balance  of each area so that the brightness in the stitched image tends to b e 
consi stent.  Different camera has different  light exposure due to brightness difference  between  
light and dark in different areas and affects the appearance . 
7. Adjust the colour  balance of the stitc hed image  which is caused by the different intensiti es 
of different channel s (B, G, R) of the camera.  
                     
Figure 27 Final image  after stitching  
 
Figure 28 Final image after brig htness bala nce

4.Stitching and smoothing of bird’s eye view  
This is the final and most important step to sti tch(merge ) all the images obtained from each 
camera to achieve bird